<a href="https://colab.research.google.com/github/AstridXiomara/LSTM/blob/main/Multivariate_LSTM_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multivariate LSTM Model

## Librerias

In [3]:
import numpy as np
import pandas as pd

# univariate lstm example
#import keras
from numpy import array
from numpy import hstack

from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Bidirectional
from keras.layers import Flatten
from keras.layers import TimeDistributed
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import ConvLSTM2D
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from sklearn.metrics import mean_squared_error

## Data

In [6]:
Constrains = pd.read_excel("Well Data Var.xlsx", sheet_name="Hoja1")
Constrains

,Pwf,Pwf2,Pwf3,Pwf4,Qw
0,3391,3286,3545,3541,1438
1,3516,3574,3297,3362,1170
2,3918,3468,3340,3085,1722
3,3972,3516,3396,3978,1671
4,3711,3546,3807,3773,548
5,3911,3758,3019,3709,967
6,3536,3678,3840,3530,818
7,3409,3275,3389,3831,1057
8,3114,3592,3436,3084,1138
9,3827,3809,3371,3766,1595


In [8]:
Production=pd.read_csv("Hoja1_results.csv", header=0, index_col=0)
Production

,0,1,2,3,4,5,6,7
0,1009.725364,677.975588,1683.374236,571.400305,0.000000,0.000000,0.000000,0.000000
1,701.721854,416.201622,946.422570,412.233996,0.007646,0.007878,0.018913,0.003323
2,675.849776,376.334852,823.549166,383.456721,0.008396,0.009304,0.021928,0.004275
3,665.542561,356.393406,766.447214,367.399296,0.008550,0.009895,0.022844,0.004791
4,658.972904,343.600407,733.636846,355.941247,0.008567,0.010208,0.023145,0.005146
...,...,...,...,...,...,...,...,...
356,104.020717,72.356329,124.298593,180.799889,442.098991,65.154675,425.571438,0.931140
357,103.619260,71.999299,123.654413,180.136330,441.037166,65.006586,425.178772,1.009742
358,103.230162,71.657921,123.033685,179.477546,440.088982,64.880222,424.825421,1.092265
359,102.852404,71.330698,122.434750,178.825734,439.239938,64.771734,424.502688,1.178638


In [46]:
Constrains2 = Constrains.copy(deep = True)
Constrains2['new_index'] = Constrains.index *15
Constrains2.set_index('new_index', inplace= True)


In [55]:
join = Constrains2.join(Production, how = 'inner')


In [68]:
data=join.to_numpy()
data

array([[3.39100000e+03, 3.28600000e+03, 3.54500000e+03, 3.54100000e+03,
        1.43800000e+03, 1.00972536e+03, 6.77975588e+02, 1.68337424e+03,
        5.71400305e+02, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00],
       [3.51600000e+03, 3.57400000e+03, 3.29700000e+03, 3.36200000e+03,
        1.17000000e+03, 5.70918516e+02, 2.31898717e+02, 7.97543919e+02,
        3.46285673e+02, 7.43352080e-03, 7.92927979e-03, 2.82565193e-02,
        7.27737804e-03],
       [3.91800000e+03, 3.46800000e+03, 3.34000000e+03, 3.08500000e+03,
        1.72200000e+03, 3.60469808e+02, 2.60930138e+02, 5.80721992e+02,
        3.51937395e+02, 5.55001916e-03, 8.29522195e-03, 2.73524215e-02,
        1.05637160e-02],
       [3.97200000e+03, 3.51600000e+03, 3.39600000e+03, 3.97800000e+03,
        1.67100000e+03, 4.47214593e+02, 2.51667978e+02, 6.04774850e+02,
        1.22392607e+02, 3.14953806e-03, 7.46684480e-03, 2.43576345e-02,
        4.08043344e-03],
       [3.71100000e+03, 3.54600000e+

In [69]:
bhp_1 = data[:,0]
bhp_2 = data[:,1]
bhp_3 = data[:,2]
bhp_4 = data[:,3]
Qw    = data[:,4]
oil_1 = data[:,5]
oil_2 = data[:,6]
oil_3 = data[:,7]
oil_4 = data[:,8]
wat_1 = data[:,9]
wat_2 = data[:,10]
wat_3 = data[:,11]
wat_4 = data[:,12]

## Funciones

In [71]:
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps):
	X, y = list(), list()
	for i in range(len(sequences)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the dataset
		if end_ix > len(sequences):
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)

In [73]:
# Modelo

def modelo(n_features, LSTM_model, x_train, y_train):
  if LSTM_model=="Vanilla":
    # define model
    model = Sequential()
    #Number of LSTM in hidden layer=50
    model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features)))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
  elif LSTM_model=="Stacked":
    # define model
    model = Sequential()
    model.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(n_steps, n_features)))
    model.add(LSTM(50, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
  elif LSTM_model=="Bidirectional":
    # define model
    model = Sequential()
    model.add(Bidirectional(LSTM(50, activation='relu'), input_shape=(n_steps, n_features)))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
  # fit model
  model.fit(x_train, y_train, epochs=200, verbose=0)

  return model

In [74]:
from numpy.ma.core import sqrt
# Prediccion

def prediccion (x_test, y_test, model):
  # demonstrate prediction
  x_input = array(x_test)
  x_input = x_input.reshape((1, n_steps, n_features))
  yhat = model.predict(x_input, verbose=0)
  RMSE = sqrt(mean_squared_error( y_test, yhat ))
  return yhat, RMSE